In [1]:
import requests
import json

In [7]:
league_id = "992211363579363328"
url = f"https://api.sleeper.app/v1/league/{league_id}/users"

response = requests.get(url)

In [21]:
data = response.json()
data

[{'user_id': '419619117067141120',
  'settings': None,
  'metadata': {'mention_pn': 'on',
   'mascot_message_emotion_leg_10': 'idle',
   'mascot_item_type_id_leg_18': 'pizza',
   'mascot_item_type_id_leg_17': 'pizza',
   'mascot_item_type_id_leg_16': 'pizza',
   'mascot_item_type_id_leg_15': 'pizza',
   'mascot_item_type_id_leg_14': 'pizza',
   'mascot_item_type_id_leg_13': 'pizza',
   'mascot_item_type_id_leg_12': 'pizza',
   'mascot_item_type_id_leg_11': 'pizza',
   'mascot_item_type_id_leg_10': 'pizza',
   'allow_pn': 'on'},
  'league_id': '992211363579363328',
  'is_owner': False,
  'is_bot': False,
  'display_name': 'TheWetCookie',
  'avatar': '87e4024cae674468adaa3cd9327fca68'},
 {'user_id': '467718102394597376',
  'settings': None,
  'metadata': {'mention_pn': 'on',
   'mascot_message_emotion_leg_5': 'taunting',
   'mascot_message_emotion_leg_1': 'idle',
   'mascot_item_type_id_leg_9': 'furrari',
   'mascot_item_type_id_leg_8': 'furrari',
   'mascot_item_type_id_leg_7': 'furrari

In [14]:
import pandas as pd

pd.read_json(data)

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [22]:
for item in data:
    del item["metadata"]

In [29]:
pd.read_json(json.dumps(data))

,user_id,settings,league_id,is_owner,is_bot,display_name,avatar
0,419619117067141120,NaN,992211363579363328,False,False,TheWetCookie,87e4024cae674468adaa3cd9327fca68
1,467718102394597376,NaN,992211363579363328,True,False,notnickwright,3849eff2b0923224ae851062a6f66b0c
2,467724601913241600,NaN,992211363579363328,False,False,ADSkinher,17a9d4efd6fc0acf39c53f9c1900dff9
3,467739982753165312,NaN,992211363579363328,False,False,jboyfresh,50ba16db60e2e064b58bc354ce163ddd
4,467844003077812224,NaN,992211363579363328,False,False,fakelymistakely,634a609edec6356659c01d1b65620bf9
5,467860779001966592,NaN,992211363579363328,False,False,30feralhogs,59fb1149ad4ecdc6b96a2feead3a07c1
6,574000543068106752,NaN,992211363579363328,False,False,Giants6and1,81dbc946fb05c852df87d5650e0e26d5
7,602280605933178880,NaN,992211363579363328,False,False,Dens1girl,db780845820808ef24d8766eae8fffdf
8,602553278961614848,NaN,992211363579363328,False,False,kimfused,8aafd2e52c63ca0a4e994c9059fd8418
9,728396625118097408,NaN,992211363579363328,False,False,DownfieldDaks,9b8f723e5213999dc0115d0ccc05edee


In [63]:
class SleeperLeague:
    # remember to incorporate json_normalize when rosters are set
    def __init__(self, league_id: int, json: bool = False):
        self.league_id = str(league_id)
        self.json = json


    def get_data(self, url):
        response = requests.get(url)
        if not self.json:
            data = json.dumps(response.json())
            return pd.read_json(data)
        else:
            return response.json()
    
    def users(self):
        url = f"https://api.sleeper.app/v1/league/{self.league_id}/users"
        users = self.get_data(url)
        return users
    
    def rosters(self):
        url = f"https://api.sleeper.app/v1/league/{self.league_id}/rosters"
        rosters = self.get_data(url)
        return rosters

In [44]:
nick_league = SleeperLeague(league_id=992211363579363328)

In [50]:
nick_league.json = False
nick_league.users()

,user_id,settings,metadata,league_id,is_owner,is_bot,display_name,avatar
0,419619117067141120,NaN,"{'mention_pn': 'on', 'mascot_message_emotion_l...",992211363579363328,0.0,False,TheWetCookie,87e4024cae674468adaa3cd9327fca68
1,467718102394597376,NaN,"{'mention_pn': 'on', 'mascot_message_emotion_l...",992211363579363328,1.0,False,notnickwright,3849eff2b0923224ae851062a6f66b0c
2,467724601913241600,NaN,"{'team_name': 'Fionn's Chins', 'mention_pn': '...",992211363579363328,0.0,False,ADSkinher,17a9d4efd6fc0acf39c53f9c1900dff9
3,467739982753165312,NaN,"{'team_name': 'Anti-Semantics', 'mention_pn': ...",992211363579363328,0.0,False,jboyfresh,50ba16db60e2e064b58bc354ce163ddd
4,467844003077812224,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",992211363579363328,0.0,False,fakelymistakely,634a609edec6356659c01d1b65620bf9
5,467860779001966592,NaN,"{'mention_pn': 'on', 'avatar': 'https://sleepe...",992211363579363328,0.0,False,30feralhogs,59fb1149ad4ecdc6b96a2feead3a07c1
6,574000543068106752,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",992211363579363328,0.0,False,Giants6and1,81dbc946fb05c852df87d5650e0e26d5
7,602280605933178880,NaN,"{'team_name': 'GiGi's Gang Green', 'show_masco...",992211363579363328,0.0,False,Dens1girl,db780845820808ef24d8766eae8fffdf
8,602553278961614848,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",992211363579363328,0.0,False,kimfused,8aafd2e52c63ca0a4e994c9059fd8418
9,728396625118097408,NaN,"{'mention_pn': 'on', 'mascot_message_emotion_l...",992211363579363328,0.0,False,DownfieldDaks,9b8f723e5213999dc0115d0ccc05edee


In [64]:
mooch_league = SleeperLeague(league_id=991871455543562240)

In [67]:
mooch_league.users()

,user_id,settings,metadata,league_id,is_owner,is_bot,display_name,avatar
0,419619117067141120,NaN,"{'team_name': 'Alex's Deadly Dad Dick', 'menti...",991871455543562240,NaN,False,TheWetCookie,87e4024cae674468adaa3cd9327fca68
1,460463959602884608,NaN,"{'team_name': 'Mooch is large', 'mention_pn': ...",991871455543562240,NaN,False,RaRaRulon,4f32bc74abe145dfe56c9f8c37433e82
2,467844003077812224,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",991871455543562240,NaN,False,fakelymistakely,634a609edec6356659c01d1b65620bf9
3,587037449620856832,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",991871455543562240,NaN,False,Alexsbigolehog,67afb25d6a1e59a718c8cfad54b1483e
4,723211408543682560,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",991871455543562240,NaN,False,Wolfmandolorian,83a04e6311b00537ce0a9931c7f35abb
5,991844287509848064,NaN,"{'team_name': 'Deshaun’s cumback story', 'ment...",991871455543562240,1.0,False,Mooch07,97d0f36ef03d4228faf12642f137c927
6,992099402997112832,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",991871455543562240,NaN,False,JGtheSmall,15000e0cd84e3cbce81aea93cd439e6d
7,992135085895868416,NaN,"{'team_name': 'MidNight Green', 'mention_pn': ...",991871455543562240,NaN,False,Jpersechino,51741071d33c395cb946bd13f8dca02d
8,992182639589146624,NaN,"{'team_name': 'Going Dyno', 'mention_pn': 'on'...",991871455543562240,NaN,False,CHughes92,8eb8f8bf999945d523f2c4033f70473e
9,992197118687248384,NaN,"{'mention_pn': 'on', 'allow_sms': 'on', 'allow...",991871455543562240,NaN,False,tmass6,e7af4deab0289b4f5505646424895246
